In [ ]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 59.7 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Parametri del problema
n = 3  # Numero di lavori
m = 3  # Numero di macchine
p = [[2, 5, 2],  # Tempi di processamento per ogni lavoro su ogni macchina
     [4, 1, 3],
     [3, 2, 5]]

# Creazione del modello
model = gp.Model("FlowShopNoBuffer")

# Variabili di decisione
C = model.addVars(n, m, vtype=GRB.CONTINUOUS, name="C") # Tempo di completamento
S = model.addVars(n, m, vtype=GRB.CONTINUOUS, name="S") # Tempo di inizio
Cmax = model.addVar(vtype=GRB.CONTINUOUS, name="Cmax") # Makespan

# Funzione obiettivo
model.setObjective(Cmax, GRB.MINIMIZE)

# Vincoli

# Vincolo di makespan
for i in range(n):
    model.addConstr(C[i, m-1] <= Cmax, "Makespan")

# Vincolo di precedenza
for i in range(n):
    for j in range(m - 1):
        model.addConstr(C[i, j] + p[i][j+1] <= C[i, j+1], "Precedenza")

# Vincolo di non sovrapposizione (con variabili binarie)
x = model.addVars(n, n, m, vtype=GRB.BINARY, name="x")
M = sum(sum(p[i]) for i in range(n)) # Big-M
for i in range(n):
    for k in range(n):
        if i != k:
            for j in range(m):
                model.addConstr(C[i, j] <= S[k, j] + M * (1 - x[i, k, j]), "NonSovrapposizione1")
                model.addConstr(C[k, j] <= S[i, j] + M * x[i, k, j], "NonSovrapposizione2")

# Vincolo di tempo di inizio e completamento
for i in range(n):
    for j in range(m):
        model.addConstr(S[i, j] + p[i][j] == C[i, j], "Tempo")



# Risoluzione del modello
model.optimize()

# Stampa dei risultati
if model.status == GRB.OPTIMAL:
    print(f"Makespan: {Cmax.X}")
    for i in range(n):
        for j in range(m):
            print(f"Lavoro {i+1} Macchina {j+1}: Inizio {S[i,j].X}, Completamento {C[i,j].X}")
else:
    print("Nessuna soluzione trovata")

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 54 rows, 46 columns and 144 nonzeros
Model fingerprint: 0x7e6dcc9e
Variable types: 19 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 19.0000000
Presolve removed 9 rows and 18 columns
Presolve time: 0.00s
Presolved: 45 rows, 28 columns, 126 nonzeros
Variable types: 10 continuous, 18 integer (18 binary)

Root relaxation: objective 1.000000e+01, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dep

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Parametri del problema
n = 3  # Numero di lavori
m = 3  # Numero di macchine
p = [[2, 5, 2],  # Tempi di processamento per ogni lavoro su ogni macchina
     [4, 1, 3],
     [3, 2, 5]]

# Creazione del modello
model = gp.Model("FlowShopNoBuffer")

# Variabili di decisione
d = model.addVars(m + 2, n, vtype=GRB.CONTINUOUS, name="d")
x = model.addVars(n, n, vtype=GRB.BINARY, name="x")

# Funzione obiettivo
model.setObjective(d[m, n - 1], GRB.MINIMIZE)

# Vincoli
for i in range(n):
    for k in range(1, m + 1):
        model.addConstr(
            d[k, i] >= d[k - 1, i] + sum(p[j][k - 1] * x[j, i] for j in range(n)),
            f"Vincolo 7.5_{i}_{k}",
        )
        if i>0:
          model.addConstr(d[k, i] >= d[k + 1, i - 1], f"Vincolo 7.6_{i}_{k}")

for j in range(n):
    model.addConstr(sum(x[i, j] for i in range(n)) == 1, f"Vincolo 7.7_{j}")

for i in range(n):
    model.addConstr(sum(x[i, j] for j in range(n)) == 1, f"Vincolo 7.8_{i}")

for i in range(n):
    for j in range(n):
        model.addConstr(x[i, j] >= 0, f"Vincolo 7.9_{i}_{j}")
        model.addConstr(x[i, j] <= 1, f"Vincolo 7.9_{i}_{j}")

for i in range(n):
    for j in range(n):
        if i > 0:
            model.addConstr(d[0, i] == d[1, i - 1], f"Vincolo 7.10_{i}")
        model.addConstr(d[m + 1, j] == 0, f"Vincolo 7.10_{j}")

# Risoluzione del modello
model.optimize()

# Stampa dei risultati
if model.status == GRB.OPTIMAL:
    print(f"Valore ottimo: {model.objVal}")
    for i in range(n):
        for k in range(m):
            print(f"d[{k+1},{i}]: {d[k+1,i].X}")
    for i in range(n):
        for j in range(n):
            if x[i, j].X > 0.5:
                print(f"Lavoro {i+1} assegnato alla posizione {j+1}")
else:
    print("Nessuna soluzione trovata")

# Stampa i risultati
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective value: {model.objVal}")
    for j in range(n):
        for i in range(m):
            job = [i for i in range(n) if x[i,j].X > 0.5][0]
            start_time = d[i,j].X
            completion_time = start_time + p[job][i]
            release_time = d[i+1,j].X
            print(f"Job {job+1}, Machine {i+1}, Start Time: {start_time}, Completion Time: {completion_time}, Release: {release_time}")
else:
    print("Nessuna soluzione trovata")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 54 rows, 24 columns and 114 nonzeros
Model fingerprint: 0x777af2f0
Variable types: 15 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 41 rows and 12 columns
Presolve time: 0.00s
Presolved: 13 rows, 12 columns, 45 nonzeros
Variable types: 0 continuous, 12 integer (9 binary)
Found heuristic solution: objective 15.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 15 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.500000000000e+01, best bound 1.500000000000e+01, gap 